In [70]:
import pandas as pd
import numpy as np
import geopandas as gp
import pickle
import datetime
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RandomizedLasso
from sklearn.linear_model import (Ridge, Lasso)
from sklearn.feature_selection import RFECV, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
#from minepy import MINE
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [53]:
# Attempting feature selection from zip code data
df = pd.read_csv('outputs/pluto_fdny_dob_census_to_zipcode.csv')
df.fillna(0, inplace=True)

# unit area of 11359 has +inf value. removing row from data
df = df[~df['ZipCode'].isin([ 11359, 0])]

# removing columns not needed for model building
for i in ['geometry', 'AREA', 'total_gas_incidents', 'ZipCode']:
    del df[i]


# standardizing all columns to be between 0 and 1
for i in ['age', 'unit_area', 'value_per_ft', 'total_population', 'total_households', 'zip_household_mean_income', 'total_units']:
    df[i] = df[i]/max(df[i])

X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values
names = df.columns.tolist()[:-1]

In [52]:
#Run a Correlation Matrix

df.corr()

,age,bldg_class_A0,bldg_class_A1,bldg_class_A2,bldg_class_A3,bldg_class_A4,bldg_class_A5,bldg_class_A6,bldg_class_A7,bldg_class_A8,...,total_population,total_households,WHITE_ratio,BLACK_AFRICAN_AMERICAN_ratio,AMERICAN_INDIAN_AND_ALASKA_NATIVE_ratio,ASIAN_ratio,NATIVE_HAWAIIAN_AND_OTHER_PACIFIC_ISLANDER_ratio,SOME_OTHER_RACE_ratio,zip_household_mean_income,gas_incidents_per_bldg_unit
age,1.000000,-0.164963,-0.157117,-0.299548,-0.104618,0.343395,-0.362214,-0.169113,0.037433,0.040001,...,0.290319,0.385090,-0.063633,0.033850,0.121689,-0.068896,0.101465,0.084763,-0.017948,0.171673
bldg_class_A0,-0.164963,1.000000,0.465718,0.541729,0.147881,-0.107715,0.147195,0.003909,-0.038807,-0.024185,...,-0.156663,-0.201336,-0.078392,-0.004752,-0.055756,0.358462,0.028643,-0.138343,-0.021542,0.110203
bldg_class_A1,-0.157117,0.465718,1.000000,0.504572,0.294532,-0.231348,0.278633,0.160124,-0.048940,-0.054807,...,-0.098773,-0.224911,-0.178016,0.143463,0.013843,0.230205,0.126188,-0.103034,-0.186786,0.004769
bldg_class_A2,-0.299548,0.541729,0.504572,1.000000,0.131220,-0.161130,0.167682,0.220957,-0.010441,0.026702,...,-0.154894,-0.222083,-0.030059,0.050513,-0.163176,0.173134,-0.071209,-0.205098,-0.055817,0.056554
bldg_class_A3,-0.104618,0.147881,0.294532,0.131220,1.000000,-0.084859,0.171323,0.019210,0.054369,-0.021227,...,-0.132681,-0.127558,0.117094,-0.146886,-0.102773,0.098506,-0.038259,-0.120328,-0.029726,-0.031027
bldg_class_A4,0.343395,-0.107715,-0.231348,-0.161130,-0.084859,1.000000,-0.173779,-0.059067,0.097306,-0.024177,...,0.053511,0.222451,0.212170,-0.001837,-0.134502,-0.161729,0.018324,-0.195851,0.340539,-0.083979
bldg_class_A5,-0.362214,0.147195,0.278633,0.167682,0.171323,-0.173779,1.000000,0.091619,-0.088162,-0.053270,...,0.104645,0.030726,0.025719,0.047863,0.030867,0.109673,-0.037435,-0.104883,-0.254347,0.065211
bldg_class_A6,-0.169113,0.003909,0.160124,0.220957,0.019210,-0.059067,0.091619,1.000000,-0.022484,-0.013212,...,-0.067688,-0.085012,0.141400,-0.011914,-0.099522,-0.127975,-0.055358,-0.079524,-0.032403,0.093134
bldg_class_A7,0.037433,-0.038807,-0.048940,-0.010441,0.054369,0.097306,-0.088162,-0.022484,1.000000,-0.010265,...,-0.150151,-0.129052,-0.103066,-0.100698,-0.130751,-0.104692,-0.049051,-0.112617,-0.084292,-0.158484
bldg_class_A8,0.040001,-0.024185,-0.054807,0.026702,-0.021227,-0.024177,-0.053270,-0.013212,-0.010265,1.000000,...,-0.118114,-0.122065,0.141385,-0.061135,-0.086761,-0.078516,-0.039959,-0.065798,0.020673,0.205139


Randomized Lasso Calculation

In [63]:
random.seed(123)
rlasso = RandomizedLasso()
rlasso.fit(X, Y)
 
print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 10000), rlasso.scores_), names), reverse=True)
#All scores equal to zero. Try another model

/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call

Features sorted by their score:
[(0.34, 'DOB_dispos_G8'), (0.285, 'DOB_permit_EQ'), (0.07, 'ECB_violation_Construction'), (0.025, 'BLACK_AFRICAN_AMERICAN_ratio'), (0.02, 'DOB_dispos_A8'), (0.01, 'bldg_class_Q2'), (0.01, 'ECB_infraction_B6A'), (0.01, 'DOB_complaint_58'), (0.005, 'DOB_violation_E-ELEVATOR'), (0.0, 'zip_household_mean_income'), (0.0, 'value_per_ft'), (0.0, 'unit_area'), (0.0, 'total_units'), (0.0, 'total_population'), (0.0, 'total_households'), (0.0, 'retail_ratio'), (0.0, 'res_unit_ratio'), (0.0, 'res_ratio'), (0.0, 'office_ratio'), (0.0, 'landuse_11'), (0.0, 'landuse_10'), (0.0, 'landuse_09'), (0.0, 'landuse_08'), (0.0, 'landuse_07'), (0.0, 'landuse_06'), (0.0, 'landuse_05'), (0.0, 'landuse_04'), (0.0, 'landuse_03'), (0.0, 'landuse_02'), (0.0, 'landuse_01'), (0.0, 'com_ratio'), (0.0, 'bldg_class_Z9'), (0.0, 'bldg_class_Z8'), (0.0, 'bldg_class_Z7'), (0.0, 'bldg_class_Z5'), (0.0, 'bldg_class_Z4'), (0.0, 'bldg_class_Z3'), (0.0, 'bldg_class_Z2'), (0.0, 'bldg_class_Z1'), (0.

/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):
/opt/rh/anaconda/root/lib/python2.7/site-packages/sklearn/linear_model/randomized_l1.py:53: DeprecationWarning: This function is deprecated. Please call randint(0, 1 + 1) instead
  for _ in range(n_resampling)):


RFE - Recursive Feature Elimination

In [68]:
#use linear regression as the model
lr = LinearRegression()
#rank all features, i.e continue the elimination until the last ten
selector = RFECV(lr, step = 1)
selector.fit(X,Y)
 
print "Features sorted by their rank:"
print sorted(zip(map(lambda x: round(x, 4), selector.ranking_), names))

Features sorted by their rank:
[(1.0, 'bldg_class_P2'), (2.0, 'ECB_infraction_B1R'), (3.0, 'DOB_dispos_D6'), (4.0, 'bldg_class_RD'), (5.0, 'bldg_class_Q2'), (6.0, 'ECB_infraction_239'), (7.0, 'DOB_dispos_EA'), (8.0, 'bldg_class_Z1'), (9.0, 'bldg_class_W5'), (10.0, 'DOB_dispos_A6'), (11.0, 'ECB_infraction_B4C'), (12.0, 'DOB_complaint_62'), (13.0, 'ECB_infraction_B8H'), (14.0, 'ECB_infraction_274'), (15.0, 'DOB_dispos_C8'), (16.0, 'DOB_dispos_C7'), (17.0, 'ECB_infraction_B13'), (18.0, 'ECB_infraction_BB0'), (19.0, 'ECB_infraction_118'), (20.0, 'ECB_infraction_B57'), (21.0, 'ECB_infraction_127'), (22.0, 'ECB_infraction_266'), (23.0, 'ECB_infraction_B70'), (24.0, 'bldg_class_RX'), (25.0, 'bldg_class_A8'), (26.0, 'bldg_class_Y4'), (27.0, 'bldg_class_Y7'), (28.0, 'DOB_violation_LL1080-LOCAL LAW 10/80 - FACADE'), (29.0, 'bldg_class_O8'), (30.0, 'ECB_infraction_BQ3'), (31.0, 'ECB_infraction_BQ2'), (32.0, 'bldg_class_U7'), (33.0, 'DOB_complaint_2C'), (34.0, 'DOB_complaint_06'), (35.0, 'ECB_infr